In [1]:
import google.generativeai as genai
import os

genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

In [2]:
def get_completion(prompt, model_name='gemini-1.5-flash'):
    messages = [{"role": "user", "parts": [prompt]}]
    model = genai.GenerativeModel(model_name)
    response = model.generate_content(messages,
                                 generation_config=genai.types.GenerationConfig(temperature=0)
                                 )
    return response.text

def get_completion_from_messages(messages, model_name='gemini-1.5-flash', temperature=0):
    model = genai.GenerativeModel(model_name)
    response = model.generate_content(messages,
                                 generation_config=genai.types.GenerationConfig(temperature=temperature)
                                 )
    return response.text

In [11]:
import gradio as gr

context = [ {'role':'model', 'parts':["""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""]} ]  # accumulate messages

def collect_messages(prompt, history):
    context.append({'role': 'user', 'parts': [f"{prompt}"]})
    response = get_completion_from_messages(context)  # This assumes you have a function to generate responses
    context.append({'role': 'model', 'parts': [f"{response}"]})

    return response


demo = gr.ChatInterface(collect_messages,
                 type="messages",
                 title="OrderBot",
                 description="Hello I am OrderBot, an automated service that collect orders for our pizza restaurant",
                 theme="soft",)
demo.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [12]:
messages =  context.copy()
messages.append(
    {'role':'model', 'parts':['create a json summary of the previous food order. Itemize the price for each item\
     The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price ']}
)
response = get_completion_from_messages(messages, temperature=0)
print(response)

```json
{
  "pizza": [
    {
      "type": "eggplant",
      "size": "small",
      "price": 6.75
    }
  ],
  "toppings": [],
  "drinks": [
    {
      "type": "coke",
      "size": "small",
      "price": 1.00
    },
    {
      "type": "coke",
      "size": "small",
      "price": 1.00
    }
  ],
  "sides": [
    {
      "type": "fries",
      "size": "large",
      "price": 4.50
    },
    {
      "type": "fries",
      "size": "large",
      "price": 4.50
    }
  ],
  "totalPrice": 18.75
}
```



In [8]:
print(messages)

[{'role': 'model', 'parts': ["\nYou are OrderBot, an automated service to collect orders for a pizza restaurant. You first greet the customer, then collects the order, and then asks if it's a pickup or delivery. You wait to collect the entire order, then summarize it and check for a final time if the customer wants to add anything else. If it's a delivery, you ask for an address. Finally you collect the payment.Make sure to clarify all options, extras and sizes to uniquely identify the item from the menu.You respond in a short, very conversational friendly style. The menu includes pepperoni pizza  12.95, 10.00, 7.00 cheese pizza   10.95, 9.25, 6.50 eggplant pizza   11.95, 9.75, 6.75 fries 4.50, 3.50 greek salad 7.25 Toppings: extra cheese 2.00, mushrooms 1.50 sausage 3.00 canadian bacon 3.50 AI sauce 1.50 peppers 1.00 Drinks: coke 3.00, 2.00, 1.00 sprite 3.00, 2.00, 1.00 bottled water 5.00 "]}, {'role': 'user', 'parts': ['helloo :)']}, {'role': 'model', 'parts': ['Hey there!  Ready to 

In [13]:
model = genai.GenerativeModel('gemini-1.5-flash')
model.count_tokens(messages)

total_tokens: 790